## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

В колонках датасета содержатся следующие данные:

- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

# Подготовка данных

In [ ]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[0]) == 3:
    from pyspark.ml.feature import OneHotEncoder as Encoder    
elif int(pyspark_version[0]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator as Encoder

RANDOM_SEED = 2022

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California Housing") \
                    .getOrCreate()

In [3]:
data = spark.read.load('/datasets/housing.csv', format="csv", sep=",", inferSchema=True, header="true")
data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
data.describe().toPandas() 

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [6]:
for column in data.columns:
    print(column, data.filter(data[column].isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [8]:
int(data.approxQuantile('total_bedrooms', [0.5], 0)[0])

435

In [9]:
data = data.na.fill({'total_bedrooms': 435})

In [10]:
for column in data.columns:
    print(column, data.filter(data[column].isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [11]:
data_train, data_test = data.randomSplit([.8,.2], seed=RANDOM_SEED)
print(data_train.count(), data_test.count())

16418 4222


В первую очередь трансформируем категориальный признак с помощью трансформера StringIndexer.

In [12]:
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx'], handleInvalid='keep').fit(data_train) 

data_train = indexer.transform(data_train)
data_test = indexer.transform(data_test)

data_train.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    1194.0|     465.0|       2.5179|           79000.0|     NEAR OCEAN|                2.0|
+---------+--------+------------------+-----------+-

OHE-кодирование 

In [13]:
encoder = Encoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe']).fit(data_train)

data_train = encoder.transform(data_train)
data_test = encoder.transform(data_test)
data_train.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    1194.0|     465.0|       2.5179|        

объединение признаков в один вектор.

In [14]:
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'],
                                        outputCol="categorical_features")

data_train = categorical_assembler.transform(data_train)
data_test = categorical_assembler.transform(data_test)
data_train.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|  -1

Трансформация числовых признаков.

In [16]:
numerical_cols  = ['longitude', 'latitude', 'housing_median_age',
                   'total_rooms', 'total_bedrooms', 'population',
                   'households', 'median_income']

In [17]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, 
                                      outputCol="numerical_features")

data_train = numerical_assembler.transform(data_train)
data_test = numerical_assembler.transform(data_test)

data_train.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|[-124.35,40.54,52...|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0

In [18]:
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled").fit(data_train)

data_train = standardScaler.transform(data_train)
data_test = standardScaler.transform(data_test)

data_train.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|numerical_features_scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|[-124.35,40.54,52...|     [-61.952887791441...|
|   -124.3|    4

In [19]:
data_train.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'ocean_proximity_idx',
 'ocean_proximity_ohe',
 'categorical_features',
 'numerical_features',
 'numerical_features_scaled']

Собирем трансформированные категорийные и числовые признаки с помощью VectorAssembler.

In [20]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features_all") 

data_train = final_assembler.transform(data_train)
data_test = final_assembler.transform(data_test)

In [21]:
numerical_features = ['numerical_features_scaled']

numerical_final_assembler = VectorAssembler(inputCols=numerical_features, 
                                  outputCol='features_numerical')

data_train = numerical_final_assembler.transform(data_train)
data_test = numerical_final_assembler.transform(data_test)

Разделение на выборки.

# Обучение моделей

In [22]:
lr = LinearRegression(labelCol='median_house_value', featuresCol='features_all')

model_all = lr.fit(data_train) 
predictions_all = model_all.transform(data_test)

24/05/04 20:09:48 WARN Instrumentation: [2a726840] regParam is zero, which might cause numerical instability and overfitting.
24/05/04 20:09:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/05/04 20:09:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/05/04 20:09:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/05/04 20:09:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
24/05/04 20:09:49 WARN Instrumentation: [2a726840] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
24/05/04 20:09:50 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
24/05/04 20:09:50 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed


In [23]:
r2_all = RegressionEvaluator(labelCol='median_house_value',metricName='r2').evaluate(predictions_all.select("median_house_value", "prediction"))
mae_all = RegressionEvaluator(labelCol='median_house_value',metricName='mae').evaluate(predictions_all.select("median_house_value", "prediction"))
rmse_all = RegressionEvaluator(labelCol='median_house_value',metricName='rmse').evaluate(predictions_all.select("median_house_value", "prediction"))
print(f'метрика R2: {r2_all}')
print(f'метрика MAE: {mae_all}')
print(f'метрика RMSE: {rmse_all}')

метрика R2: 0.6535582201164198
метрика MAE: 49867.04607181885
метрика RMSE: 68486.7300501659


In [24]:
lr = LinearRegression(labelCol='median_house_value', featuresCol='features_numerical')

model_numerical = lr.fit(data_train) 
predictions_numerical = model_numerical.transform(data_test)

24/05/04 20:09:54 WARN Instrumentation: [ef411830] regParam is zero, which might cause numerical instability and overfitting.


In [25]:
r2_numerical = RegressionEvaluator(labelCol='median_house_value',metricName='r2').evaluate(predictions_numerical.select("median_house_value", "prediction"))
mae_numerical = RegressionEvaluator(labelCol='median_house_value',metricName='mae').evaluate(predictions_numerical.select("median_house_value", "prediction"))
rmse_numerical = RegressionEvaluator(labelCol='median_house_value',metricName='rmse').evaluate(predictions_numerical.select("median_house_value", "prediction"))
print(f'метрика R2: {r2_numerical}')
print(f'метрика MAE: {mae_numerical}')
print(f'метрика RMSE: {rmse_numerical}')

метрика R2: 0.6461729709922107
метрика MAE: 50866.55977336322
метрика RMSE: 69212.86125298042


In [26]:
spark.stop()

# Анализ результатов

Как мы видим, отличия при использовании разных данных минимальны.\
Значения метрик в обоих случаях довольно близки, что говорит о том, что модель, построенная на всех данных, показывает практически такие же результаты, как и модель, построенная только на числовых переменных.\
\
Вывод: Обе модели показывают примерно одинаковые результаты, и использование всех доступных переменных (как числовых, так и категориальных) позволяет построить немного более качественную модель, чем использование только числовых переменных.